# Data Analytics Learning Framework Entry Test #

## Introduction ##

Sandra is a manufacturer of widgets. She has 5 colours of widgets: Blue, Red, Orange, Green and Purple. Recently she has had a issue with quality of the parts related to lengths the widgets being outside of tolerance. Each colour group has an associated quality check tolerence band for the lengths. She has asked her team to measure the length of the most recent batch of parts and mark down the colour of each.

She has turned to you to help her analyse the data. She has provided you will two datasets:
- The measurements & colours per part taken by her team.
- The tolerences for each colour group. (Note that the tolerences are inclusive)

For each colour group she would like to know:
- Number of parts in the group
- Median length value (to 3 decimal places e.g. 99.999)
- Interquartile range (to 3 decimal places e.g. 9.999)
- Percentage of parts that pass their corresponding tolerence band (to 2 decimal places e.g 99.99%)

Note: Unfortunately while marking down the colour of each part her team used slightly different syntax to describe the colour. She would like you to correct this before producing her outputs. -> DONE

# Set up #

### Do not modify the code in the next box ###

Execute the code below to get your data.

In [1]:
#### DO NOT MODIFY ###

import pandas as pd
import numpy as np

seeding = 0

R_mu, R_sig = (16.0, 0.5)
B_mu, B_sig = (16.0, 0.2)
G_mu, G_sig = (16.4, 0.4)
P_mu, P_sig = (16.0, 0.3)
O_mu, O_sig = (16.8, 0.8)

df_config = pd.DataFrame({
    "Colour_raw":['RED','R','Red','red','BLUE','Bleu','Blue','B','G','Green','GREEN','green','O','Orang','Orange','PURP','Purpal','Purple'],
    "Colour_True" : ['Red','Red','Red','Red','Blue','Blue','Blue','Blue','Green','Green','Green','Green','Orange','Orange','Orange','Purple','Purple','Purple'],
    "N_parts" : [2301,5325,6003,8002,1200,490,4450,1151,1990,9012,342,153,5515,23,6351,741,698,4900],
    "Mu": [R_mu,R_mu,R_mu,R_mu,B_mu,B_mu,B_mu,B_mu,G_mu,G_mu,G_mu,G_mu,O_mu,O_mu,O_mu,P_mu,P_mu,P_mu],
    "Sig": [R_sig,R_sig,R_sig,R_sig,B_sig,B_sig,B_sig,B_sig,G_sig,G_sig,G_sig,G_sig,O_sig,O_sig,O_sig,P_sig,P_sig,P_sig]

})

df_mast = pd.DataFrame()
for n1 in range(0,len(df_config)):
    np.random.seed(seeding)
    df_local = pd.DataFrame(
        {'Colour':df_config.loc[n1,'Colour_raw'],
        'Length':np.round(np.random.normal(df_config.loc[n1,'Mu'],df_config.loc[n1,'Sig'],df_config.loc[n1,'N_parts']),3)})
    df_mast = pd.concat([df_mast,df_local])

Measurements = df_mast.sample(frac=1).reset_index(drop=True)
    
Tolerences = pd.DataFrame({'Colour':['Red','Blue','Green','Orange','Purple'],
                       'Lower Tolerence':[14.50,15.625,15.60,14.15,15.25],
                       'Upper Tolerence':[19.150,17.755,18.25,17.95,17.25]})


## Instructions ##

Your two data sets are:

Measurements  -  Which is the list of measurements of length of widgets measured by the team

Tolerences    -  The upper and lower tolerence linked to each colour category

## Measurement dataset ##

Below is a sample of the Measurements data set. As you can see the team have not been consistend in the way they label colour. However the length has consistently been given to 3 decimal places

In [78]:
from scipy.stats import iqr

In [3]:
Measurements.head(5)

,Colour,Length
0,Red,16.096
1,Orange,17.613
2,Green,16.008
3,Orange,18.189
4,Orange,17.376


In [7]:
len(Measurements)

58647

In [7]:
Measurements.tail(5)

,Colour,Length
58642,Purple,15.636
58643,R,15.880
58644,BLUE,15.891
58645,red,16.806
58646,Blue,16.050


In [16]:
Measurements["Colour"].head(10)

0       Red
1    Orange
2     Green
3    Orange
4    Orange
5     Green
6       Red
7     Green
8     Green
9       Red
Name: Colour, dtype: object

In [14]:
Measurements.replace(to_replace="((^R|^r)[A-z]*)", value="Red", regex=True, inplace=True)
Measurements.replace(to_replace="((^O|^o)[A-z]*)", value="Orange", regex=True, inplace=True)
Measurements.replace(to_replace="((^B|^b)[A-z]*)", value="Blue", regex=True, inplace=True)
Measurements.replace(to_replace="((^G|^g)[A-z]*)", value="Green", regex=True, inplace=True)
Measurements.replace(to_replace="((^P|^p)[A-z]*)", value="Purple", regex=True, inplace=True)

## Tolerence dataset ##

In [7]:
Tolerences

,Colour,Lower Tolerence,Upper Tolerence
0,Red,14.500,19.150
1,Blue,15.625,17.755
2,Green,15.600,18.250
3,Orange,14.150,17.950
4,Purple,15.250,17.250


In [170]:
# create output dataframe
results=pd.DataFrame(columns=["Number of Parts", "Median Length", "Interquartile Range", "Percentage Tolerence Bounds Pass"],
                    index=["Red", "Blue", "Green", "Orange", "Purple"])

In [176]:
# Fill number of Parts
results["Number of Parts"]=Measurements["Colour"].value_counts()

In [196]:
# Fill everything else
for index in results.index.values:
    lengths=Measurements[Measurements["Colour"]==index]["Length"]
    tolerences=Tolerences[Tolerences["Colour"]==index][["Lower Tolerence", "Upper Tolerence"]]
    results.loc[index, "Median Length"]=round(lengths.median(),3)
    # WARNING: Maybe, I should not use scipy as it rounds differently than the expected answer
    #   I should do it by myself
    results.loc[index, "Interquartile Range"]=round(iqr(lengths),3)
    results.loc[index, "Percentage Tolerence Bounds Pass"]=round(lengths[(lengths>=tolerences["Lower Tolerence"].iloc[0]) & (lengths<=tolerences["Upper Tolerence"].iloc[0])].count()/results.loc[index, "Number of Parts"],4)

In [197]:
results

,Number of Parts,Median Length,Interquartile Range,Percentage Tolerence Bounds Pass
Red,21631,15.989,0.669,0.9990
Blue,7291,15.993,0.267,0.9734
Green,11497,16.39,0.531,0.9791
Orange,11889,16.784,1.075,0.9272
Purple,6339,15.988,0.403,0.9931


In [170]:
results=pd.DataFrame(columns=["Number of Parts", "Median Length", "Interquartile Range", "Percentage Tolerence Bounds Pass"],
                    index=["Red", "Blue", "Green", "Orange", "Purple"])

In [191]:
results

,Number of Parts,Median Length,Interquartile Range,Percentage Tolerence Bounds Pass
Red,21631,15.989,0.669,0.998983
Blue,7291,15.993,0.267,0.972980
Green,11497,16.39,0.531,0.979038
Orange,11889,16.784,1.075,0.927075
Purple,6339,15.988,0.403,0.993059


In [165]:
Measurements["Colour"].value_counts()

Red       21631
Orange    11889
Green     11497
Blue       7291
Purple     6339
Name: Colour, dtype: int64

In [176]:
# Fill number of Parts
results["Number of Parts"]=Measurements["Colour"].value_counts()

In [190]:
# Fill Median Length
for index in results.index.values:
    lengths=Measurements[Measurements["Colour"]==index]["Length"]
    tolerences=Tolerences[Tolerences["Colour"]==index][["Lower Tolerence", "Upper Tolerence"]]
    results.loc[index, "Median Length"]=lengths.median()
    results.loc[index, "Interquartile Range"]=iqr(lengths)
    results.loc[index, "Percentage Tolerence Bounds Pass"]=lengths[(lengths>tolerences["Lower Tolerence"].iloc[0]) & (lengths<tolerences["Upper Tolerence"].iloc[0])].count()/results.loc[index, "Number of Parts"]

In [114]:
x = Measurements[Measurements["Colour"]==index]["Length"]
x[(x<15.7) & (x>15.6)].count()

406

In [168]:
results.loc["Red", "Median Length"]

15.989

## Hints ##

### Table ###
You will want to aggregate the data into a table similar to this:

|  | Number of Parts | Median Length | Interquartile Range | Percentage Tolerence Bounds Pass |
| -- | -- | -- | -- | -- | -- |
| Blue | | | | |
| Red | | | | |
| Green | | | | |
| Orange | | | | |
| Purple | | | | | |


### Inclusive Tolerences ###

Inclusive tolerences mean that if the length of the part in question is the exactly same length as either the upper or the lower bound then it will pass the quality check.

### Median and Interquartile Range ###
Please check that you are calculating the right thing
- https://en.wikipedia.org/wiki/Median
- https://en.wikipedia.org/wiki/Interquartile_range

### Don't reinvent the wheel ###
There are plenty of python packages (namely numpy and pandas) to help you do this... 
